# Preprocessing

### Libraries Importing

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score

In [29]:
df= pd.read_csv('Cleaned_df.csv')
df.head()

,neo_id,name,absolute_magnitude,estimated_diameter_min,estimated_diameter_max,relative_velocity,miss_distance,is_hazardous
0,2162117,162117 (1998 SD15),19.14,0.394962,0.883161,71745.401048,5.814362e+07,False
1,2349507,349507 (2008 QY),18.50,0.530341,1.185878,109949.757148,5.580105e+07,True
2,2455415,455415 (2003 GA),21.45,0.136319,0.304818,24865.506798,6.720689e+07,False
3,3132126,(2002 PB),20.63,0.198863,0.444672,78890.076805,3.039644e+07,False
4,3557844,(2011 DW),22.70,0.076658,0.171412,56036.519484,6.311863e+07,False


#### Drop unvaluable columns

In [30]:
df.nunique()

neo_id                     33511
name                       33511
absolute_magnitude          1778
estimated_diameter_min      1778
estimated_diameter_max      1778
relative_velocity         338161
miss_distance             337798
is_hazardous                   2
dtype: int64

In [31]:
#dropping columns with high collinearity with absolute_magnitude
df.drop(columns = ['estimated_diameter_min','estimated_diameter_max'], inplace= True)

#dropping columns with high dimentionality
df.drop(columns = ['neo_id','name'], inplace= True)


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338171 entries, 0 to 338170
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   absolute_magnitude  338171 non-null  float64
 1   relative_velocity   338171 non-null  float64
 2   miss_distance       338171 non-null  float64
 3   is_hazardous        338171 non-null  bool   
dtypes: bool(1), float64(3)
memory usage: 8.1 MB


#### Split the Data to taregt and features

In [33]:
X = df.drop(columns = ['is_hazardous'])
print(X.shape)
y=df['is_hazardous']
print(y.shape)

(338171, 3)
(338171,)


#### Target Encoding

Since our target datatype is bolean, we don't need to encode it.
We can replance the encoding step with converting the datatype to int.

In [34]:
y = y.astype('int')
y.unique()

array([0, 1])

#### Train test split

To prevent data leakage from the test set, we need to split the data into training and testing sets before proceeding with the preprocessing.

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(270536, 3)
(270536,)
(67635, 3)
(67635,)


#### Handling Target Imbalance


Here we chose to apply SMOTE on the training set only to keep the testing set unseen. also the test set should be from the real world so should be always balanced

In [36]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train,y_train)

#### Scaling the numeric data


To make sure that all features are contributing in the training equally without one dominating others, we need to standardize the numeric features.

In [37]:
#instantiate the scaler
scaler = StandardScaler()

#Fit and transform the scaler to the training data
scaled_X_train = scaler.fit_transform(X_resampled)

#use the same scaler for the test data

scaled_X_test = scaler.transform(X_test)

In [38]:
scaled_X_test

array([[ 1.57659883, -1.33014028, -1.56224825],
       [ 1.72735993,  1.00548846,  0.79021598],
       [ 1.31653593, -1.48586428, -0.74730354],
       ...,
       [ 1.3127669 , -1.30853803, -0.53471346],
       [ 1.463528  , -1.12324679, -1.83569083],
       [-1.29916916,  1.26965119, -0.24130895]])

In [39]:
#Convert back to DataFrame
X_train_scaled = pd.DataFrame( scaled_X_train , columns=X_train.columns)
print(X_train_scaled.head())
X_test_scaled = pd.DataFrame( scaled_X_test , columns=X_train.columns)
print(X_train_scaled.head())

   absolute_magnitude  relative_velocity  miss_distance
0           -1.174791           2.701825       0.999402
1            0.351665          -0.657867      -1.155717
2            1.900735          -1.010679       0.322506
3            0.148137           0.658436       1.414836
4            1.388147          -0.673429       0.121050
   absolute_magnitude  relative_velocity  miss_distance
0           -1.174791           2.701825       0.999402
1            0.351665          -0.657867      -1.155717
2            1.900735          -1.010679       0.322506
3            0.148137           0.658436       1.414836
4            1.388147          -0.673429       0.121050


# Model Building

#### Model Training

In [40]:
# instantiate the model
rf_model = RandomForestClassifier(n_estimators=50, random_state=42)

# Train the model
rf_model.fit(X_train_scaled, y_resampled)

RandomForestClassifier(n_estimators=50, random_state=42)

In [41]:
# Make predictions
y_pred = rf_model.predict(X_test_scaled)

In [42]:
# Accuracy Score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.8248

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.82      0.89     59088
           1       0.41      0.87      0.56      8547

    accuracy                           0.82     67635
   macro avg       0.69      0.84      0.72     67635
weighted avg       0.91      0.82      0.85     67635


Confusion Matrix:
[[48352 10736]
 [ 1111  7436]]


In [43]:
y_proba = rf_model.predict_proba(X_test_scaled)[:, 1]
print(f"AUC-ROC: {roc_auc_score(y_test, y_proba):.4f}")

AUC-ROC: 0.9206


In [47]:
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [20, 30],
}

random_search = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=param_grid,
    n_iter=10,
    cv=3,
    scoring='recall',  
    n_jobs=-1,
    random_state=42
)

random_search.fit(X_train_scaled, y_resampled)

RandomizedSearchCV(cv=3,
                   estimator=RandomForestClassifier(n_estimators=50,
                                                    random_state=42),
                   n_jobs=-1,
                   param_distributions={'max_depth': [None, 10, 20],
                                        'min_samples_split': [20, 30],
                                        'n_estimators': [100, 200]},
                   random_state=42, scoring='recall')

In [48]:
best_rf = random_search.best_estimator_
best_rf.fit(X_train_scaled, y_resampled)

RandomForestClassifier(max_depth=10, min_samples_split=20, random_state=42)

In [51]:
random_search.best_score_

0.9863556518899564

In [52]:
y_proba = random_search.predict_proba(X_test_scaled)[:, 1]
print(f"AUC-ROC: {roc_auc_score(y_test, y_proba):.4f}")

AUC-ROC: 0.8861


# Model Deployment

In [ ]:
import joblib

# Save the trained model
joblib.dump(rf_model, 'random_forest_model.pkl')
